In [5]:
from flask import Flask, Response
import json, os

app = Flask(__name__)

# ---------- serve the possession JSON ----------
@app.route("/sequence.json")
def sequence_json():
    path = os.path.join(os.path.dirname(__file__), "sequence.json")
    if not os.path.exists(path):
        return Response(json.dumps({"error": "sequence.json not found"}), status=404, mimetype="application/json")
    with open(path, "r", encoding="utf-8") as f:
        data = f.read()
    # pass-through (already valid JSON)
    return Response(data, mimetype="application/json")

# ---------- main page: crisp pitch + labeled arrows ----------
@app.route("/focus")
def focus():
    html = r"""<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8"/><meta name="viewport" content="width=device-width,initial-scale=1"/>
<title>xG Pitch (focus)</title>
<style>
  :root { --bg:#0b0d10; --card:#101317; --grid:#445063; --home:#60a5fa; --away:#f472b6; --muted:#a6b0bf; }
  *{box-sizing:border-box}
  body{margin:0;background:var(--bg);color:#e7ecf2;font-family:Inter,system-ui,Segoe UI,Roboto,Arial}
  .wrap{max-width:1200px;margin:22px auto;padding:0 18px}
  .top{display:flex;justify-content:space-between;align-items:center;margin-bottom:10px}
  .kpi{font-size:14px;color:#d5dde7}
  button{background:#1a2130;border:1px solid #2a3242;border-radius:10px;color:#e7ecf2;padding:6px 12px;cursor:pointer}
  button:hover{background:#222a3a}
  .card{background:var(--card);border-radius:16px;box-shadow:0 10px 30px rgba(0,0,0,.35);padding:12px}
  svg{width:100%;min-height:560px;height:auto;display:block}
</style>
</head>
<body>
<div class="wrap">
  <div class="top">
    <div class="kpi" id="kpi">Cumulative xG: –</div>
    <button id="reload">Reload JSON</button>
  </div>
  <div class="card">
    <svg id="pitch" viewBox="-3 -3 126 86" aria-label="Football pitch"></svg>
  </div>
</div>

<script>
const $ = s => document.querySelector(s);
const pitch = $('#pitch'), kpi = $('#kpi');

function n(tag){return document.createElementNS('http://www.w3.org/2000/svg',tag)}
function rect(p,x,y,w,h,stroke='#445063',wth=1.2){const e=n('rect');e.setAttribute('x',x);e.setAttribute('y',y);e.setAttribute('width',w);e.setAttribute('height',h);e.setAttribute('fill','none');e.setAttribute('stroke',stroke);e.setAttribute('stroke-width',wth);p.appendChild(e)}
function line(p,x1,y1,x2,y2,stroke='#445063',w=1.2,op=.95){const e=n('line');e.setAttribute('x1',x1);e.setAttribute('y1',y1);e.setAttribute('x2',x2);e.setAttribute('y2',y2);e.setAttribute('stroke',stroke);e.setAttribute('stroke-width',w);e.setAttribute('opacity',op);e.setAttribute('stroke-linecap','round');p.appendChild(e)}
function arrow(p,x1,y1,x2,y2,stroke,w=1.9,op=.96){line(p,x1,y1,x2,y2,stroke,w,op);const a=Math.atan2(y2-y1,x2-x1),L=3;const ax=x2-L*Math.cos(a-Math.PI/8),ay=y2-L*Math.sin(a-Math.PI/8),bx=x2-L*Math.cos(a+Math.PI/8),by=y2-L*Math.sin(a+Math.PI/8);const h=n('polyline');h.setAttribute('points',`${ax},${ay} ${x2},${y2} ${bx},${by}`);h.setAttribute('fill','none');h.setAttribute('stroke',stroke);h.setAttribute('stroke-width',w);h.setAttribute('opacity',op);h.setAttribute('stroke-linecap','round');h.setAttribute('stroke-linejoin','round');p.appendChild(h)}
function dot(p,cx,cy,r,fill,op=.95){const e=n('circle');e.setAttribute('cx',cx);e.setAttribute('cy',cy);e.setAttribute('r',r);e.setAttribute('fill',fill);e.setAttribute('opacity',op);p.appendChild(e)}
function outlineText(p,x,y,txt,size=8.5){const t=n('text');t.setAttribute('x',x.toFixed(2));t.setAttribute('y',y.toFixed(2));t.setAttribute('font-size',size);t.setAttribute('fill','#e7ecf2');t.setAttribute('paint-order','stroke');t.setAttribute('stroke','#0b0d10');t.setAttribute('stroke-width','2');t.textContent=txt;p.appendChild(t)}
function labelOnArrow(p,x1,y1,x2,y2,text){const ang=Math.atan2(y2-y1,x2-x1),mx=(x1+x2)/2,my=(y1+y2)/2,ox=4*Math.sin(ang),oy=-4*Math.cos(ang);outlineText(p,mx+ox,my+oy,text)}

function drawBasePitch(){
  pitch.innerHTML='';
  const g=n('g');
  // boundary + halfway + circle
  rect(g,0,0,120,80,'#52607a',1.6);
  line(g,60,0,60,80,'#4b5970',1.4);
  const cc=n('circle'); cc.setAttribute('cx',60); cc.setAttribute('cy',40); cc.setAttribute('r',9.15);
  cc.setAttribute('fill','none'); cc.setAttribute('stroke','#4b5970'); cc.setAttribute('stroke-width','1.4'); g.appendChild(cc);
  // penalty & 6yd boxes
  rect(g,0,18,18,44); rect(g,0,30,6,20);
  rect(g,102,18,18,44); rect(g,114,30,6,20);
  // goals (thin)
  rect(g,-1.5,36,1.5,8,'#3f4a5d',1.1); rect(g,120,36,1.5,8,'#3f4a5d',1.1);
  pitch.appendChild(g);
}

// --- scaling/orientation ---
function detectScale(data){
  let m = 0;
  data.forEach(e=>{m=Math.max(m, e.x||0,e.y||0,e.end_x||0,e.end_y||0)});
  if (m <= 1.05) return {sx:120, sy:80};     // 0..1
  if (m <= 100) return {sx:1.2, sy:0.8};     // 0..100
  return {sx:1, sy:1};                       // already 120x80
}
function scaleEvent(e, sx, sy){return {x:(+e.x||0)*sx, y:(+e.y||0)*sy, ex:(+e.end_x||+e.x||0)*sx, ey:(+e.end_y||+e.y||0)*sy}}
function flipX(ev){return {x:120-ev.x, y:ev.y, ex:120-ev.ex, ey:ev.ey}}

const isShot = e => ((e.event||'')+'').toLowerCase().includes('shot');
const lastShotIdx = arr => { for (let i=arr.length-1;i>=0;i--) if (isShot(arr[i])) return i; return -1; };

function normalize(data){
  const {sx,sy} = detectScale(data);
  const norm = data.map((d,i)=>{
    const s = scaleEvent(d, sx, sy);
    const team = ((d.team||'home')+'').toLowerCase()==='away' ? 'away' : 'home';
    const color = getComputedStyle(document.documentElement).getPropertyValue(team==='away'?'--away':'--home').trim();
    return {i:i+1, event:(d.event||'event'), x:s.x, y:s.y, ex:s.ex, ey:s.ey, xg:+(d.xg||0), color};
  });
  // auto-orient: ensure final shot attacks to the RIGHT
  const sidx = lastShotIdx(norm);
  if (sidx >= 0 && norm[sidx].ex < 60) {
    for (let j=0;j<norm.length;j++){ norm[j] = {...norm[j], ...flipX(norm[j])}; }
  }
  return norm;
}

function draw(data){
  drawBasePitch();
  const evs = normalize(data);
  const cum = (data||[]).reduce((s,e)=>s+(+e.xg||0),0);

  const g = n('g'); pitch.appendChild(g);
  evs.forEach(e=>{
    arrow(g, e.x, e.y, e.ex, e.ey, e.color, 2.0, .98);
    const r = 2.6 + 8*Math.sqrt(Math.min(e.xg,1));
    dot(g, e.ex, e.ey, r, e.color, .96);
    labelOnArrow(g, e.x, e.y, e.ex, e.ey, `#${e.i} · ${e.event}`);
  });

  const sidx = lastShotIdx(evs);
  if (sidx >= 0){
    const s = evs[sidx];
    const ring = n('circle');
    ring.setAttribute('cx', s.ex); ring.setAttribute('cy', s.ey);
    ring.setAttribute('r', (3.2 + 8*Math.sqrt(Math.min(s.xg,1))).toFixed(2));
    ring.setAttribute('fill','none'); ring.setAttribute('stroke','#f8cc4a'); ring.setAttribute('stroke-width','2.2');
    pitch.appendChild(ring);
  }

  kpi.textContent = `Cumulative xG: ${cum.toFixed(2)}`;
}

async function load(){
  drawBasePitch();
  const r = await fetch('/sequence.json?ts='+Date.now());
  const txt = await r.text(); let data;
  try { data = JSON.parse(txt); } catch(e){ kpi.textContent = 'JSON parse error'; return; }
  if (!Array.isArray(data)){ kpi.textContent = 'Bad JSON'; return; }
  draw(data);
}

document.getElementById('reload').onclick = load;
load();
</script>
</body>
</html>
"""
    return Response(html, mimetype="text/html")


# quick health-check
@app.route("/ping")
def ping(): return "pong"

if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
